In [88]:
import asyncio
import aiohttp
import aiomoex
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import nest_asyncio


nest_asyncio.apply()

#get a stock list from MOEX exchange
async def get_stock_list():
    request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/boards/TQBR/securities.json"
    arguments = {"securities.columns": ("SECID," "REGNUMBER," "LOTSIZE," "SHORTNAME")}
    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, arguments)
        data = await iss.get()
        data = pd.DataFrame(data["securities"])
        return data['SECID']

#get a stock list from MOEX exchange
async def get_stock_list_with_extra_data():
    request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/boards/TQBR/securities.json"
    arguments = {"securities.columns": ("SECID," "REGNUMBER," "LOTSIZE," "SHORTNAME")}
    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, arguments)
        data = await iss.get()
        data = pd.DataFrame(data["securities"])
        return data

#get a price history for the ticker in TQBR regime    
async def get_price_history(ticker):
    async with aiohttp.ClientSession() as session:
        data = await aiomoex.get_board_history(session, ticker)
        df = pd.DataFrame(data)
        df['TICKER'] = ticker
        return df

#get a price history for the ticker in TQBR regime  
async def get_multiple_price_histories(tickers):
    tasks = [get_price_history(ticker) for ticker in tickers]
    results = await asyncio.gather(*tasks)
    combined_df = pd.concat(results, ignore_index=True)
    combined_df['TRADEDATE'] = pd.to_datetime(combined_df['TRADEDATE'])
    condition = (combined_df['TICKER'] == 'GMKN') & (combined_df['TRADEDATE'] <= '2024-04-01')
    combined_df.loc[condition, 'CLOSE'] = combined_df.loc[condition, 'CLOSE'] / 100
    condition = (combined_df['TICKER'] == 'TRNFP') & (combined_df['TRADEDATE'] <= '2024-02-14')
    combined_df.loc[condition, 'CLOSE'] = combined_df.loc[condition, 'CLOSE'] / 100
    return combined_df


#check the stocks which are trading near min for the last n years
def get_tickers_which_we_can_buy_now(df, years, percent):

    # Ensure TRADEDATE is in datetime format
    df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
    # Calculate date n years ago
    today = datetime.now()
    n_years_ago = today - timedelta(days=years*365)
    
    # Filter data for the last n years
    df_last_n_years = df[df['TRADEDATE'] >= n_years_ago]
    
    # Calculate the minimum and maximum closing price for each ticker over the last 2 years
    min_prices = df_last_n_years.groupby('TICKER')['CLOSE'].min().reset_index()
    min_prices.rename(columns={'CLOSE': 'MIN_CLOSE'}, inplace=True)
    max_prices = df_last_n_years.groupby('TICKER')['CLOSE'].max().reset_index()
    max_prices.rename(columns={'CLOSE': 'MAX_CLOSE'}, inplace=True)
    
    # Get the latest closing price for each ticker
    latest_prices = df_last_n_years.sort_values('TRADEDATE').groupby('TICKER').tail(1).reset_index()
    latest_prices = latest_prices[['TICKER', 'CLOSE']].rename(columns={'CLOSE': 'LATEST_CLOSE'})
    
    # Merge the minimum, maximum, and latest prices
    merged_df = pd.merge(min_prices, max_prices, on='TICKER')
    merged_df = pd.merge(merged_df, latest_prices, on='TICKER')
    
    # Filter tickers where the latest closing price is not higher than x% above the minimum price
    condition = merged_df['LATEST_CLOSE'] <= merged_df['MIN_CLOSE'] * (1+percent/100)
    result = merged_df[condition]
    
    return result

#remove duplicates from table
def remove_duplicates(database_path, table_name):
    """
    Connects to an SQLite database and removes duplicate rows from the specified table.

    Parameters:
    database_path (str): The path to the SQLite database file.
    table_name (str): The name of the table from which to remove duplicates.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Define the query to remove duplicates
    delete_duplicates_query = f"""
    WITH duplicates AS (
        SELECT 
            ROWID,
            BOARDID,
            TRADEDATE,
            CLOSE,
            VOLUME,
            VALUE,
            TICKER,
            ROW_NUMBER() OVER (PARTITION BY BOARDID, TRADEDATE, TICKER ORDER BY ROWID) AS rn
        FROM 
            {table_name}
    )
    DELETE FROM {table_name}
    WHERE ROWID IN (
        SELECT ROWID
        FROM duplicates
        WHERE rn > 1
    );
    """

    try:
        # Execute the query to delete duplicates
        cursor.execute(delete_duplicates_query)
        # Commit the transaction
        conn.commit()
    except sqlite3.Error as error:
        print(f"Error while removing duplicates: {error}")
    finally:
        # Close the connection
        conn.close()



def insert_data_to_database(database_name, table_name, df):
    conn = sqlite3.connect(database_name)
    df.to_sql(table_name, conn, if_exists='append', index=False)
    remove_duplicates(database_name,table_name)
    conn.close()
    

In [89]:
stock_list = asyncio.run(get_stock_list())
stock_list_extra_data = asyncio.run(get_stock_list_with_extra_data())
price_history = asyncio.run(get_price_history('GAZP'))
combined_df = asyncio.run(get_multiple_price_histories(stock_list))
pivot_df = combined_df.pivot(index='TRADEDATE', columns='TICKER', values='CLOSE')
filered_df = get_tickers_which_we_can_buy_now(combined_df,3,10)
insert_data_to_database('/Users/antonvolkov/Desktop/MoexTrading/MoexData.db','moexdata',combined_df)